In [2]:
# !pip install transformers datasets torch

In [3]:
import pandas as pd
import json
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,DatasetDict


In [4]:
# data = pd.read_excel('/content/qa-pairs.xlsx')
data = pd.read_excel('./proj/combiner/latest - after peft data/qa-pairs-3.xlsx')


In [5]:
data = data[['instruction', 'response']]

In [6]:
# data.to_csv("qa_data.csv", index=False)
data.to_csv("./proj/combiner/latest - after peft data/qa-pairs-3_cs", index=False)

In [7]:
from datasets import load_dataset

# dataset = load_dataset("csv", data_files="qa_data.csv")
dataset = load_dataset("csv", data_files="./proj/combiner/latest - after peft data/qa-pairs-3_cs")


Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)


train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']

In [9]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")
# Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

c:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\.cache\huggingface\hub\models--google--mt5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [10]:
# import torch

# def preprocess_function(examples):
#     # Tokenize each instruction and response pair
#     inputs = examples["instruction"]
#     targets = examples["response"]

#     # Handle None values in both inputs and targets lists
#     inputs = [q if q is not None else "" for q in inputs]
#     targets = [a if a is not None else "" for a in targets]

#     # Tokenize inputs and targets
#     model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

#     # Tokenize targets separately
#     labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

#     # Assign tokenized labels
#     model_inputs["labels"] = labels

#     # Make sure all tensors in model_inputs are contiguous
#     for k, v in model_inputs.items():
#         if isinstance(v, torch.Tensor) and not v.is_contiguous():
#             model_inputs[k] = v.contiguous()

#     return model_inputs

In [24]:
import torch
def preprocess_function(examples):
    # Tokenize each instruction and response pair with prompt design

    inputs = examples["instruction"]
    targets = examples["response"]

    # Handle None values in both inputs and targets lists
    inputs = [q if q is not None else "" for q in inputs]
    targets = [a if a is not None else "" for a in targets]

    # Add prompt design: prepend "Question:" to inputs and "Answer:" to outputs
    formatted_inputs = ["Question: " + q + " Answer:" for q in inputs]

    # Tokenize inputs and targets with the tokenizer
    model_inputs = tokenizer(formatted_inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets separately
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    # Assign tokenized labels to the model inputs
    model_inputs["labels"] = labels

    # Make sure all tensors are contiguous
    for k, v in model_inputs.items():
        if isinstance(v, torch.Tensor) and not v.is_contiguous():
            model_inputs[k] = v.contiguous()

    return model_inputs


In [11]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_valid = valid_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6987 [00:00<?, ? examples/s]

Map:   0%|          | 0/873 [00:00<?, ? examples/s]

Map:   0%|          | 0/874 [00:00<?, ? examples/s]

In [1]:
# !pip install accelerate

In [13]:
import gc
from transformers import Trainer, TrainingArguments,TrainerCallback


In [14]:
import torch

# Make sure model weights are contiguous
def make_contiguous(model):
    for param in model.parameters():
        if param.data.is_contiguous() is False:
            param.data = param.data.contiguous()

# Ensure model weights are contiguous before training
make_contiguous(model)


In [15]:
# class ClearCacheCallback(TrainerCallback):
#     def on_epoch_begin(self, args, state, control, **kwargs):
#         """
#         Event called at the beginning of an epoch.
#         """
#         gc.collect()
torch.cuda.empty_cache()


In [16]:
import tempfile
# with tempfile.TemporaryDirectory() as temp_dir:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=2000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.678000,0.903282
2,0.807500,0.469814
3,0.658200,0.450345


TrainOutput(global_step=5241, training_loss=3.290410776034586, metrics={'train_runtime': 3758.6632, 'train_samples_per_second': 5.577, 'train_steps_per_second': 1.394, 'total_flos': 6283301491703808.0, 'train_loss': 3.290410776034586, 'epoch': 3.0})

In [18]:
# Save the model and tokenizer
model.save_pretrained("./trained_mt5_model")
tokenizer.save_pretrained("./trained_mt5_model")


('./trained_mt5_model/tokenizer_config.json',
 './trained_mt5_model/special_tokens_map.json',
 './trained_mt5_model/spiece.model',
 './trained_mt5_model/added_tokens.json')

In [9]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# Load the model and tokenizer from the saved path
model = MT5ForConditionalGeneration.from_pretrained("./trained_mt5_model")
tokenizer = MT5Tokenizer.from_pretrained("./trained_mt5_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available


OSError: Incorrect path_or_model_id: './trained_mt5_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# def generate_response(instruction, model, tokenizer):
#     # Tokenize the instruction
#     inputs = tokenizer(instruction, return_tensors="pt", padding=True, truncation=True).to(device)

#     # Generate output (with the model)
#     output = model.generate(input_ids=inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

#     # Decode the output token IDs back into text
#     response = tokenizer.decode(output[0], skip_special_tokens=True)
#     return response



In [21]:
sample_inputs = valid_dataset["instruction"][:7]  # Get a few sample inputs

# Tokenize the inputs
inputs = tokenizer(sample_inputs, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate the model's predictions
with torch.no_grad():  # Disable gradients for inference
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

# Decode the predictions and print the results
generated_responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Show the original input and corresponding generated responses
for input_text, generated_text in zip(sample_inputs, generated_responses):
    print(f"Input: {input_text}")
    print(f"Generated Response: {generated_text}")
    print("="*50)

Input: ਸਾਇਰਾ ਖਾਨ ਨੇ ਆਪਣਾ ਘਰ ਕਿਉਂ ਛੱਡਿਆ?
Generated Response: ਉਸਨੇ ਆਪਣਾ ਘਰ ਛੱਡਿਆ।
Input: ਪ੍ਰਸਿੱਧ ਰਾਗ ਧੰਨਾ ਸਿਰੀ ਦਾ ਕਰਤਾ ਕੌਣ ਹੈ?
Generated Response: ਉਸ ਦਾ ਕਰਤਾ ਕੌਣ ਹੈ।
Input: Year in which Sohan Singh Herreh was featured as a Living Legend in Ophthalmology by the Indian Journal of Ophthalmology?
Generated Response: Amar Singh Herreh was featured as a Living Legend in Ophthalmology by the Indian Journal of Ophthalmology.
Input: What awards did Yami Gautam win for her performance in Vicky Donor?
Generated Response: awards for her performance in Vicky Donor.
Input: ਲੈਲਾ ਵਸਤੀ ਦੇ ਪਿਤਾ ਦਾ ਨਾਮ ਕੀ ਸੀ?
Generated Response: ਉਸ ਦੇ ਪਿਤਾ ਦਾ ਨਾਮ ਕੀ ਸੀ।
Input: Where did Suba Singh complete his matriculation?
Generated Response: Gurdeep Singh complete his matriculation at the University of Delhi.
Input: What is culture transformation?
Generated Response: Gurgaon, Punjab, India


# FOR PEFT BELOW #
 

In [1]:
import pandas as pd
import json
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,DatasetDict


In [3]:
# data = pd.read_excel('/content/qa-pairs.xlsx')
# data = pd.read_excel('./proj/combiner/latest - after peft data/qa-pairs-3.xlsx')
data = pd.read_excel('./proj/combiner/latest - after peft data/qa-punjb_aigment.xlsx')

In [4]:
data = data[['instruction', 'response']]

In [6]:
data.to_csv("./qa-pairs_punj_only.csv", index=False)

In [7]:
from datasets import load_dataset

# dataset = load_dataset("csv", data_files="qa_data.csv")
# dataset = load_dataset("csv", data_files="./qa-pairs-3_cs.csv")
dataset = load_dataset("csv", data_files="./qa-pairs_punj_only.csv")


Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
# from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model = MT5ForConditionalGeneration.from_pretrained("./base_model")
# model = AutoModelForSeq2SeqLM.from_pretrained("./proj/base_mt5")
model = AutoModelForSeq2SeqLM.from_pretrained("./peft_punj/base")
tokenizer = AutoTokenizer.from_pretrained("./peft_punj/base")
# tokenizer = MT5Tokenizer.from_pretrained("./base_model/tokenizer")
# Load model directly

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

Some weights of the model checkpoint at ./peft_punj/base were not used when initializing MT5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_B.default.weight', 'decoder.block.1.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.1.layer.0.SelfAttention.q.lor

In [ ]:
# model.save_pretrained("./proj/base_mt5")
# tokenizer.save_pretrained("./proj/base_mt5")

('./proj/base_mt5\\tokenizer_config.json',
 './proj/base_mt5\\special_tokens_map.json',
 './proj/base_mt5\\spiece.model',
 './proj/base_mt5\\added_tokens.json',
 './proj/base_mt5\\tokenizer.json')

In [16]:
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)


train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']

In [17]:
from peft import PeftModel, PeftConfig

# Path to your PEFT checkpoint
# peft_checkpoint_path = "./proj/results/model7_peft_augment/checkpoint-139280"
# peft_checkpoint_path = "./proj/model7/results-model7_peft/model7_peft/checkpoint-260700"
peft_checkpoint_path = "./peft_punj/peft"

# Load the PEFT adapter configuration
# adapter_config = PeftConfig.from_pretrained(peft_checkpoint_path)
# adapter_config = PeftConfig.from_pretrained(peft_checkpoint_path)

# Load the PEFT adapter into the base model
model = PeftModel.from_pretrained(model, peft_checkpoint_path)


In [18]:
import torch

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 768)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 768)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
      

In [20]:
sample_inputs = valid_dataset["instruction"][:7]  # Get a few sample inputs
# sample_inputs = "Who is Khushwant Singh"
# Tokenize the inputs
inputs = tokenizer(sample_inputs, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate the model's predictions
with torch.no_grad():  # Disable gradients for inference
    # outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Decode the predictions and print the results
generated_responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Show the original input and corresponding generated responses
for input_text, generated_text in zip(sample_inputs, generated_responses):
    print(f"Input: {input_text}")
    print(f"Generated Response: {generated_text}")
    print("="*50)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input: ਖੁਰਸ਼ੀਦ ਬਾਨੋ ਨੂੰ ਕਦੋਂ ਸਹਿਣਾ ਪਿਆ?
Generated Response: ਉਸ
Input: ਦੇਸ ਰਾਜ ਛਾਜਲੀ ਨੇ ਕਿੱਥੇ ਪ੍ਰਦਰਸ਼ਨ ਕੀਤਾ ਹੈ?
Generated Response: ਉਸ
Input: ਸਮੀਨਾ ਪੀਰਜ਼ਾਦਾ ਨੇ ਵਿਆਹ ਤੋਂ ਬਾਅਦ ਆਪਣਾ ਨਾਂ ਡੂੰਘਾ ਕਿਉਂ ਰੱਖਿਆ?
Generated Response: ਉਸ
Input: ਸਿਪਾਹੀ/ਆਨਰੇਰੀ ਸਮੁੰਦਰੀ ਕੈਪਟਨ ਹਰਭਜਨ ਸਿੰਘ ਕਦੋਂ ਵੱਸੇ ਸਨ?
Generated Response: ਉਸ
Input: ਮੈਡੀਕੋ ਸ਼ਹਿਬਾਜ਼ ਮਲਿਕ ਦਾ ਸਥਾਨ ਕੀ ਸੀ ਜਦੋਂ ਉਸਨੇ ਪਹਿਲੀ ਪੀਐਚ.ਡੀ. ਪੱਛਮੀ_ਪਾਕਿਸਤਾਨ ਵਿੱਚ ਪੰਜਾਬੀ ਵਿੱਚ?
Generated Response: ਉਸ
Input: ਗੁਰਚੇਤ ਚਿੱਤਰਕਾਰ ਦਾ ਕਿੱਤਾ ਕੀ ਹੈ?
Generated Response: ਉਸ
Input: ਅੰਮ੍ਰਿਤਾ ਚੀਮਾ ਨੇ ਸੰਪਾਦਿਤ ਅਤੇ ਐਂਕਰ ਕੀਤੇ 60 ਮਿੰਟ ਦੇ ਪ੍ਰੋਗਰਾਮ ਦਾ ਕੀ ਨਾਮ ਹੈ?
Generated Response: ਉਸ
